In [1]:
# !pip install PyMySQL

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


# 데이터 정의
## **SQL의 분류**
* 데이터 정의어(DDL) : CREATE, ALTER, DROP
* 데이터 조작어(DML) : SELECT, INSERT, UPDATE, DETELE
* 데이터 제어어(DCL) : REVOKE, GRANT

In [3]:
import pymysql
import pandas as pd
import qgrid

In [4]:
# DB 생성
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'root',
    password = '1234',
    charset='utf8'
)

new_db = "CREATE DATABASE customer"

cursor = db.cursor()
cursor.execute(new_db)


1

---

In [5]:
# DB 연결
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = 'root',
    password = '1234',
    db = 'customer',
    charset='utf8'
)

cursor = db.cursor()

In [6]:
# 연결한 DB와 상호작용하기 위해 cursor 객체를 생성
# cursor = db.cursor() -> 일반 Cursor를 사용하면 결과가 튜플 형태로 리턴
cursor = db.cursor(pymysql.cursors.DictCursor) # 딕셔너리 형태로 리턴

---

#  데이터 정의 
* 테이블 생성 : CREATE TABLE
* 테이블 변경 : ALTER TABLE
* 테이블 제거 : DROP TABLE

## <span class="mark">**테이블 생성 : CREATE TABLE**</span>

* 고객 테이블은 고객아이디, 고객이름, 나이, 등급, 직업, 적립금 속성으로 구성되고 고객아이디 속성이 기본키이다. 고객 이름과 등급 속성은 값을 반드시 입력해야 하고, 적립금 속성은 값을 입력하지 않으면 0이 기본으로 입력되도록 고객 테이블을 생성

In [7]:
customer_sql = '''
CREATE TABLE 고객(
    고객아이디 VARCHAR(20) NOT NULL, 
    고객이름 VARCHAR(10) NOT NULL,
    나이 INT,
    등급 VARCHAR(10) NOT NULL,
    직업 VARCHAR(20),
    적립금 INT DEFAULT 0,
    PRIMARY KEY(고객아이디)
)
'''

cursor.execute(customer_sql)
db.commit()

## <span class="mark">**테이블 변경 : ALTER TABLE**</span>

[Mysql 데이터 타입 참고](https://dololak.tistory.com/260)
<br>(1) 새로운 속성 추가
<br> ALTER TABLE 테이블_이름 ADD 속성_이름 데이터_타입 [NOT NULL] [DEFAULT 기본_값];
<br><br>  - 고객 테이블에 가입날짜 속성 추가

In [8]:
customer_sql = '''
ALTER TABLE 고객 ADD 가입날짜 DATE
'''

cursor.execute(customer_sql)
db.commit()

(2) 기존의 속성 삭제
<br> ALTER TABLE 테이블_이름 DROP COLUMN 속성_이름;
<br><br> - 고객 테이블에 가입날짜 속성 삭제

In [9]:
customer_sql = '''
ALTER TABLE 고객 DROP COLUMN 가입날짜
'''

cursor.execute(customer_sql)
db.commit()

(3) 제약조건 추가
<br> ALTER TABLE 테이블_이름 ADD CONSTRAINT 제약조건_이름 제약조건_내용;
<br><br> - 고객 테이블에 20세 이상의 고객만 가입 할 수 있게 데이터 무결성 제약조건 추가
<br>[데이터 무결성 참고](https://limkydev.tistory.com/161)



In [10]:
customer_sql = '''
ALTER TABLE 고객 ADD CONSTRAINT CHK_AGE CHECK(나이>=20)
'''

cursor.execute(customer_sql)
db.commit()

(4) 제약 조건 삭제
<br> ALTER TABLE 테이블_이름 DROP CONSTRAINT 제약조건_이름;
<br><br> - 고객 테이블에 20세 이상의 고객만 가입 할 수 있는 데이터 무결성 제약조건 삭제

In [8]:
# 기존 제약 조건 삭제
customer_sql = '''
ALTER TABLE 고객 DROP CONSTRAINT CHK_AGE
'''

cursor.execute(customer_sql)
db.commit()

## <span class="mark">**테이블의 삭제 : DROP TABLE**</span>
ALTER TABLE 테이블_이름 DROP COLUMN 속성_이름; 

In [20]:
customer_sql = "DROP TABLE 고객"

cursor.execute(customer_sql)
db.commit()

---

# 데이터 조작
##  <span class="mark">**데이터 삽입 : INSERT**</span>
INSERT INTO 테이블_이름 [(속성_리스트)] VALUES (속성값_리스트);
<br><br> - 고객 테이블에 강소현 데이터 삽입

In [11]:
customer_sql = '''
INSERT INTO 고객 (고객아이디, 고객이름, 나이, 등급, 직업, 적립금) 
VALUES ('sohyun1024', '강소현', '25', 'VIP', '백수', '12000')
'''

cursor.execute(customer_sql)
db.commit()

In [12]:
# DataFrame으로 확인
customer_sql = "SELECT * FROM 고객"

cursor.execute(customer_sql)
sql_result = cursor.fetchall() # 모든 데이터를 한번에 가져올 때 사용
sql_result = pd.DataFrame(sql_result)
# sql_result
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

<br> - 고객 정보 n개 행(row)에 넣기

In [13]:
customer_sql = '''
INSERT INTO 고객 (고객아이디, 고객이름, 나이, 등급, 직업, 적립금) 
VALUES (%s, %s, %s, %s, %s, %s)
'''
val = [
    ('apple', '정소화', '20', 'GOLD', '학생', '1000'),
    ('banana', '김선우', '25', 'VIP', '간호사', '2500'),
    ('carrot', '고명석', '28', 'GOLD', '교사', '4500'),
    ('orange', '김용욱', '22', 'SILVER', '학생', '0'),
    ('melon', '성원용', '35', 'GOLD', '회사원', '5000'),
    ('peach', '오형준', '29', 'SILVER', '의사', '300'),
    ('pear', '채광주', '31', 'SILVER', '회사원', '500')
]

# executemany() : 다중 실행 함수
# executemany()함수의 매개변수로 sql과 튜플을 담은 리스트 val를 주면 됨.
cursor.executemany(customer_sql, val)
db.commit()

In [14]:
# DataFrame으로 확인
customer_sql = "SELECT * FROM 고객"

cursor.execute(customer_sql)
sql_result = cursor.fetchall() # 모든 데이터를 한번에 가져올 때 사용
sql_result = pd.DataFrame(sql_result)
# sql_result
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

<br> - 나이 제약조건 설정 확인

In [15]:
customer_sql = '''
INSERT INTO 고객 (고객아이디, 고객이름, 나이, 등급, 직업, 적립금) 
VALUES ('turtlejjang', '명기', '4', 'VVIP', '거북이', '30000')
'''

cursor.execute(customer_sql)
db.commit()

OperationalError: (3819, "Check constraint 'CHK_AGE' is violated.")

## <span class="mark">**데이터 변경 : UPDATE**</span>
UPDATE 테이블_이름 SET 속성_이름1 = 값1, 속성_이름2 = 값2 .. [WHERE 조건];
<br><br>  - 고객 테이블에 강소현 취업

In [16]:
customer_sql = '''
UPDATE 고객
SET 직업 = '댄서'
WHERE 고객아이디 = 'sohyun1024'
'''

cursor.execute(customer_sql)
db.commit()

In [17]:
# DataFrame으로 확인
customer_sql = "SELECT * FROM 고객"

cursor.execute(customer_sql)
sql_result = cursor.fetchall() # 모든 데이터를 한번에 가져올 때 사용
sql_result = pd.DataFrame(sql_result)
# sql_result
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## <span class="mark">**데이터 삭제 : DELETE**</span>
DELETE FROM 테이블_이름 [WHERE 조건];
<br><br> - 고객 테이블에 직업이 의사인 사람 데이터 삭제

In [18]:
customer_sql = '''
DELETE FROM 고객
WHERE 직업 = '의사'
'''

cursor.execute(customer_sql)
db.commit()

In [19]:
# DataFrame으로 확인
customer_sql = "SELECT * FROM 고객"

cursor.execute(customer_sql)
sql_result = cursor.fetchall() # 모든 데이터를 한번에 가져올 때 사용
sql_result = pd.DataFrame(sql_result)
# sql_result
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

##  <span class="mark">**기본 검색 : SELECT**</span>
SELECT [ALL | DISTINCT] 속성_리스트 FROM 테이블_리스트; 
<br><br> - 고객 테이블에 존재하는 모든 속성 검색

In [20]:
# * 는 ALL을 의미
customer_sql = "SELECT * FROM 고객"

cursor.execute(customer_sql)
sql_result = cursor.fetchall()
sql_result = pd.DataFrame(sql_result)
# sql_result
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

<br> - 고객 테이블에서 고객아이디, 고객이름, 등급 속성을 검색

In [21]:
customer_sql = '''
SELECT 고객아이디, 고객이름, 등급 
FROM 고객
'''

cursor.execute(customer_sql)
sql_result = cursor.fetchall()
sql_result = pd.DataFrame(sql_result)
qgrid.show_grid(sql_result)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [22]:
# DB 연결 닫기
db.close()